<a href="https://colab.research.google.com/github/AndromedaOMA/Advanced_Analytics_with_Apache_Spark---E.On_Software_Development/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descriere proiect

Acest proiect are ca scop principal analiza consumului de energie al unui grup de consumatori fictivi, pe parcursul unui an, a unei companii de energie, folosind tehnicile de analiză din motorul Apache Spark.

1. Primul set de date reflectă atât consumul total de energie, cât și detalii specifice, dacă există, despre producția din panouri solare, consumul pentru vehicule electrice (EV), energia furnizată înapoi către rețeaua electrică, consumul și încărcarea bateriilor.
2. Al doilea set de date oferă atât tariful pe an și prețurile per kWh în diferite intervale de timp, unice pentru fiecare client în parte, cât și prețul de vânzare la nivelul companiei de energie pe diferite intervale de timp.

Proiectul implică curățarea și prelucrarea datelor, completarea valorilor lipsă, iar la final calculul facturii de energie, bonus fiind compararea facturii cu alți clienți similari.



---

# Seturile de date

## Pregătire mediu de lucru

In [1]:
from google.colab import drive
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://dlcdn.apache.org/spark/
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"
import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as f
spark= SparkSession.builder.getOrCreate()
spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,542 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,140 kB]
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubu

## Setul de Date – Raw Time Series

Acest set de date conține informații despre consumul de date a unor clienți fictivi ai unei companii de energie. Structura datelor este una similară cu schemele folosite la momentul actual pentru astfel de date.

In [89]:
parquet_path = '/content/drive/MyDrive/E.on/E.on_Data/Data/Project/raw_time_series/parquet'
raw_time_series_df = spark.read.parquet(parquet_path)
raw_time_series_df.show(truncate=False)

+-------------------+-------------------+--------------------+------------+--------------------------+
|contract_id        |timestamp          |value               |value_source|annotations               |
+-------------------+-------------------+--------------------+------------+--------------------------+
|04_02_111 _ CHR12  |2023-01-01 06:00:00|0.02591860654732236 |measurement |{"region":"Europe/Berlin"}|
|04 _02_111 _CHR12  |2023-01-01 17:00:00|0.07385444264936832 |measurement |{"region":"Europe/Berlin"}|
|04_02_111 _ CHR12  |2023-01-01 17:30:22|0.08180149515221906 |measurement |{"region":"Europe/Berlin"}|
|04 _02_111 _CHR12  |2023-01-01 21:30:00|0.08670661371854547 |measurement |{"region":"Europe/Berlin"}|
|04 _ 02 _111_CHR12 |2023-01-02 00:30:00|0.03597601881331959 |measurement |{"region":"Europe/Berlin"}|
|04 _02_111 _ CHR12 |2023-01-02 05:30:00|0.03638379308965683 |measurement |{"region":"Europe/Berlin"}|
|04 _02 _111 _CHR12 |2023-01-03 10:45:00|0.931575            |measurement

## Setul de Date – Customer Tariff

Acest set de date conține informații despre tarifele și prețurile a unor clienți fictivi, într-un interval de timp, ai unei companii de energie. Structura este una similară cu cea folosite la momentul actual pentru astfel de date.

In [90]:
parquet_path = '/content/drive/MyDrive/E.on/E.on_Data/Data/Project/customer_tariff/parquet'
customer_tariff_df = spark.read.parquet(parquet_path)
customer_tariff_df.show()

+---------------+----------------------------+--------------------------+-----------------+-----------+------+
|    contract_id|target_local_start_timestamp|target_local_end_timestamp|      tariff_name|charge_type| price|
+---------------+----------------------------+--------------------------+-----------------+-----------+------+
|04_02_111_CHR28|         2022-12-01 00:00:00|       2023-02-16 00:00:00|     Electric Pro|        buy| 0.302|
|04_02_111_CHR28|         2023-02-16 00:00:00|       2024-05-19 00:00:00| Electric Loyalty|        buy|0.3938|
|04_02_111_CHR28|         2024-05-19 00:00:00|       2024-08-31 00:00:00|     Eco Electric|        buy|0.2095|
|04_02_111_CHR28|         2024-08-31 00:00:00|       2024-10-09 00:00:00| Electric Loyalty|        buy|0.4047|
|04_02_111_CHR28|         2024-10-09 00:00:00|       2024-11-09 00:00:00|Business Electric|        buy|0.1716|
|04_02_111_CHR28|         2024-11-09 00:00:00|       2024-12-11 00:00:00|     Electric Pro|        buy|0.2944|
|

---

# Cerințe

## Curățarea Datelor

Setul de date al consumului prezintă unele probleme, precum spații suplimentare și informații lipsă sau semi-structurate.

Acestea trebuie rectificate înainte de a trece mai departe:

1. Curățarea coloanei „contract_id” de spații suplimentare
2. Setarea coloanei „value_source” cu valoarea „missing” atunci când valoarea lipsește.
3. Coloana „timestamp” are variații mici ce trebuie rectificate. Contoarele măsoară consumul odată la 15 minute exact începând cu ora 00:00, dar din cauza procesării, pot apărea variații la timpul pe care îl trimit. (Indiciu: Se găsește cea mai apropriat multiplu de 15 la minute și se scot secundele)



In [91]:
# 1.
raw_time_series_df = raw_time_series_df.withColumn('contract_id', f.translate(f.col('contract_id'), ' ', ''))
raw_time_series_df.show(truncate=False)

+---------------+-------------------+--------------------+------------+--------------------------+
|contract_id    |timestamp          |value               |value_source|annotations               |
+---------------+-------------------+--------------------+------------+--------------------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:30:22|0.08180149515221906 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 00:30:00|0.03597601881331959 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 05:30:00|0.03638379308965683 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-03 10:45:00|0.931575            |measurement |{"region":"Europe/Berlin"}|
|04_02_111

In [92]:
# 2.
# raw_time_series_df = raw_time_series_df.withColumn('value_source', f.coalesce(f.col('value_source'), f.lit('missing')))
raw_time_series_df = raw_time_series_df.withColumn('value_source', f.when(f.col('value').isNull(), f.lit('missing')).otherwise(f.col('value_source')))
raw_time_series_df.show(truncate=False)

+---------------+-------------------+--------------------+------------+--------------------------+
|contract_id    |timestamp          |value               |value_source|annotations               |
+---------------+-------------------+--------------------+------------+--------------------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:30:22|0.08180149515221906 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 00:30:00|0.03597601881331959 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 05:30:00|0.03638379308965683 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-03 10:45:00|0.931575            |measurement |{"region":"Europe/Berlin"}|
|04_02_111

In [93]:
# 3.
raw_time_series_df = raw_time_series_df.withColumn('timestamp',
                                                   f.from_unixtime((f.unix_timestamp('timestamp') / 900).cast('int') * 900).cast('timestamp'))
raw_time_series_df.show(truncate=False)

+---------------+-------------------+--------------------+------------+--------------------------+
|contract_id    |timestamp          |value               |value_source|annotations               |
+---------------+-------------------+--------------------+------------+--------------------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 00:30:00|0.03597601881331959 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 05:30:00|0.03638379308965683 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-03 10:45:00|0.931575            |measurement |{"region":"Europe/Berlin"}|
|04_02_111

## Extragerea Informațiilor de localizare și filtrarea datelor invalide

Setul de date al consumului prezintă unele probleme, precum spații suplimentare și informații lipsă sau semi-structurate.

Acestea trebuie rectificate înainte de a trece mai departe:

1. Extragerea unei noi coloane „region” din „annotations” (Se recomandă folosirea funcțiilor de Spark de procesare JSON)
2. Clienții cu regiuni invalide se vor scoate din setul de date și se vor salva pe disk într-o locație separată.
3. Extragerea datei din coloana „timestamp” într-o nouă coloană „utc_date”
4. Calcularea datei locale pentru data și ora din „timestamp”, pe baza regiunii, într-o nouă coloană „local_timestamp”
5. Extragerea datei din coloana „ local_timestamp” într-o nouă coloană „local_date”



In [94]:
# 1.
raw_time_series_df = raw_time_series_df.withColumn('region', f.get_json_object(f.col('annotations'), '$.region'))
print(raw_time_series_df.count())
raw_time_series_df.show(5, truncate=False)

3549244
+---------------+-------------------+-------------------+------------+--------------------------+-------------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |
+---------------+-------------------+-------------------+------------+--------------------------+-------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-02 00:30:00|0.03597601881331959|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
+---------------+-------------------+-------------------+------------+--------------------------

In [95]:
# 2.
# Am verificat câte regiuni se găsesc în setul nostru de date -> REZULTAT: Europe/Berlin și WakaWaka
view_regions = raw_time_series_df.select('region').distinct().collect()
print(view_regions)

test_raw_time_series_df = raw_time_series_df.groupBy('region').avg()
test_raw_time_series_df.show(truncate=False)

# Așadar:
out_path = '/content/drive/MyDrive/E.on/E.on_Data/Data/Project/outputs'
invalid_regions_df = raw_time_series_df.where(f.col('region')!='Europe/Berlin')
invalid_regions_df.write.mode('overwrite').format('parquet').save(out_path)

print(f"no. of rows of raw_time_series_df: {raw_time_series_df.count()}")
raw_time_series_df.show(5, truncate=False)

print(f"no. of rows of invalid_regions_df: {invalid_regions_df.count()}")
invalid_regions_df.show(5, truncate=False)

[Row(region='Europe/Berlin'), Row(region='WakaWaka')]
+-------------+-------------------+
|region       |avg(value)         |
+-------------+-------------------+
|Europe/Berlin|0.490498974431521  |
|WakaWaka     |0.13178303122889254|
+-------------+-------------------+

no. of rows of raw_time_series_df: 3549244
+---------------+-------------------+-------------------+------------+--------------------------+-------------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |
+---------------+-------------------+-------------------+------------+--------------------------+-------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR

In [96]:
# 3.
raw_time_series_df = raw_time_series_df.withColumn('utc_date', f.to_date(f.col('timestamp')))
raw_time_series_df.show(5, truncate=False)

+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|
|04_02_111_CHR12|2023-01-02 00:30:00|0.03597601881331959|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-02|
+---------------

In [98]:
# 4.
raw_time_series_df = raw_time_series_df.withColumn('local_timestamp', f.from_utc_timestamp(f.col('timestamp'), f.col('region')))
raw_time_series_df.show(5, truncate=False)

+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 07:00:00|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 18:00:00|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 18:30:00|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 22:30:00|

In [100]:
# 5.
raw_time_series_df = raw_time_series_df.withColumn('local_date', f.to_date('local_timestamp'))
raw_time_series_df.show(5, truncate=False)

+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |local_date|
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 07:00:00|2023-01-01|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 18:00:00|2023-01-01|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 18:30:00|2023-01-01|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547|measurement |{"regi

## Extragerea Informațiilor de consum

Setul de date al consumului prezintă unele probleme, precum spații suplimentare și informații lipsă sau semi-structurate.



Acestea trebuie rectificate înainte de a trece mai departe:

1. Extragerea din coloana „ annotations” a consumului de vehicul electric (EV), baterie (BATTERY_IN) și consumul trimis spre rețeaua electrică (GRID_SELL) în coloanele „sent_to_ev”, „sent_to_battery” și „sent_to_grid”. În cazul în care valoare lipsește, se consideră consumul 0.
2. Extragerea din coloana „ annotations” a energiei primite de la panourile solare (PV) și baterie (BATTERY_OUT) în coloanele „received_from_pv” și „received_from_battery”. În cazul în care valoare lipsește, se consideră energia primită 0.

## Filtrarea consumului neobișnuit

Anumite valori ale consumului sunt neobișnuit de mari și este necesară scoaterea lor.



1.  Setarea coloanei „value_source” în „plausability_check_failed” pentru valorile cu consum neobișnuit din setul de date. (Decizia valorilor mari fie se face cu o analiză vizuală a datelor (sortarea și identificarea lor vizual fie prin agregări simple fie mai complicate) sau bonus, pentru cine dorește, prin tehnici de învățare automată.)
2. Salvarea separată a datelor cu „value_source” având valoare „plausability_check_failed” într-o locație separată. Atenție, datele nu se scot din setul de date.
3. Setarea coloanei „value” în NULL pentru datele cu „value_source” având valoare „plausability_check_failed”.



## Completarea valorilor lipsă

Anumiți clienți au lipsuri prezintă câteva lipsuri în consum, unele dintre ele adăugate de noi la pasul precedent:




1. Prezicerea valorii „value” atunci când ea este NULL folosind următoarea metodă:

  * Calculăm mai întâi media din ultimele 8 săptămâni a valorilor din aceeași zi a săptămânii la aceeași oră, minut și secundă. Se folosesc din ultimele 8 săptămâni doar valorile care nu lipsesc, au coloana „value_source” setată pe valoarea „measurement”.
  * Facem suma coloanelor „sent_to_ev”, „sent_to_battery” și „sent_to_grid”.
  * Facem suma coloanelor „received_from_pv” și „received_from_battery”.
  * Completăm coloane „value” cu maximum dintre aceste 3 valori.
  * **Bonus**, pentru cine dorește, puteți folosi și algoritmi de învățare automată, precum Linear Regression, în loc de calcularea mediei din ultimele 8 săptămâni și să comparați cele 2 metode.


2. Calcularea valorii „received_from_grid” atunci când avem toate informațiile.
  * Facem suma coloanelor „sent_to_ev”, „sent_to_battery” și „sent_to_grid” și scădem valorile din coloanele „received_from_pv” și „received_from_battery”.



## Asocierea cu tarifere

Curățând datele și completând valorile lipsă, putem acuma să trecem la asocierea cu tarifele consumatorilor:



1. Asocierea intrărilor de consum cu cele de tarifare, pe bază numărului de contract, al timpului și al tipului de preț, cumpărare sau vânzare.
  * Prețul tarifat la cumpărare este prețul de cumpărare pentru intrare de consum înmulțit cu coloana „received_from_grid”
  * Prețul tarifat la vânzare este prețul de vânzare pentru intrare de consum înmulțit cu coloana „send_to_grid”.

## Calcularea consumului și a facturii

După asociere, putem trece la calculul factorii:


1. Per zi / săptămâna / lună / an, pentru fiecare client, să se calculeze:
  * Consumul de energie total (suma coloanei value) – „ kWh_total”
  * Energia extrasă din baterie și cea din PV – „kWh_from_battery”, „kWh_from_PV”
  * Energia folosită pentru EV – „kWh_for_EV”
  * Consumul de energie folosit de la rețea – „kWh_from_grid”
  * Costul consumului de energie folosit de la rețea – „price_billed”
  * Consumul de energie trimis către rețea – „kWh_to_grid”
  * Costul primit înapoi, a consumului de energie trimis către rețea – „price_cashback”
  * Costul total (diferența de cost) – „price_final”
  * Puneți codul de calculare al agregatelor pe orice perioadă de timp într-o funcție, iar pentru fiecare interval, zi / săptămâna / lună / an, rulați această funcție. Afișați pentru fiecare perioada top 10 clienți cu cel mai mare preț.